In [1]:
import cv2 as cv
import math
import numpy as np
##角度制转弧度制
RAD = lambda v: v*math.pi/180;
##xyz方向上偏转矩阵
Rx = lambda rx: np.mat([[1,0,0],[0,math.cos(rx),-math.sin(rx)],[0,math.sin(rx),math.cos(rx)]])
Ry = lambda ry: np.mat([[math.cos(ry),0,math.sin(ry)],[0,1,0],[-math.sin(ry),0,math.cos(ry)]])
Rz = lambda rz: np.mat([[math.cos(rz),-math.sin(rz),0],[math.sin(rz),math.cos(rz),0],[0,0,1]])

In [2]:
if __name__=="__main__":
    f,cx,cy,theta=1000,320,240,math.pi/12
    cam_pos=[[0,0,0],[-2,-2,0],[2,2,0],[-2,2,0],[2,-2,0]]
    cam_ori=[[0,0,0],[-theta,theta,0],[theta,-theta,0],[theta,theta,0],[-theta,-theta,0]]
    file= open("./data/box.xyz")
    X=np.array([0,0,0,0]).reshape(1,4)
    for line in file.readlines():
        if line=="\n":
            continue
        data=np.array(line.strip("\n").split(" ")).astype(np.float)
        data=np.concatenate((data,[1]),axis=0).reshape(1,4)
        X=np.concatenate((X,data),axis=0)
    X=np.delete(np.mat(X.T),0,axis=1)
    K=np.mat([[f,0,cx],[0,f,cy],[0,0,1]])
    for i in range(len(cam_pos)):
        ##project matrix
        Rc=Rz(cam_ori[i][2])*Ry(cam_ori[i][1])*Rx(cam_ori[i][0])
        tc=np.mat(cam_pos[i]).T
        Rt=np.concatenate((Rc.T,-Rc.T*tc.reshape(3,1)),axis=1)
        P=K*Rt
        
        ##project the point
        x=P*X
        x[0]=x[0]/x[2]
        x[1]=x[1]/x[2]
        x[2]=1
        
        #gaussian noise
        noise=np.random.randn(2,x[0].shape[1])
        x[0]=x[0]+noise[0]
        x[1]=x[1]+noise[1]
        n,m=640,480
        image=np.zeros((m,n))
        for i in range(x[0].shape[1]):
            p=np.mat(x[:2,i])
            if p[0,0]>=0 and p[0,0]<=n and p[1,0]>=0 and p[1,0]<=m:
                cv.circle(image,(int(p[0,0]),int(p[1,0])),2,255,-1)
        cv.imshow("aa",image)
        cv.waitKey(0)
        cv.destroyAllWindows()

In [4]:
# print(x)
n,m=640,480
image=np.zeros((m,n))
for i in range(x[0].shape[1]):
    p=np.mat(x[:2,i])
    if p[0,0]>=0 and p[0,0]<=n and p[1,0]>=0 and p[1,0]<=m:
        cv.circle(image,(int(p[0,0]),int(p[1,0])),2,255,-1)
cv.imshow("aa",image)
cv.waitKey(0)
cv.destroyAllWindows()

In [15]:
a=x[:2,0]
a.shape

(2, 1)